In [1]:
%autoreload 2
%matplotlib inline

/home/micha/work/occrp/followthemoney-predict/experiments/xref-experiments/venv/lib/python3.8/site-packages/IPython/core/pylabtools.py:280: MatplotlibDeprecationWarning: 
The keymap.all_axes rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  import matplotlib


In [2]:
from pathlib import Path
import json

import pandas as pd

import torch
from xref_experiments.model import PropertySkipgramModel
from xref_experiments.create_model_data import load_vocabularies, ParquetModelData

In [3]:
import pylab as py
import seaborn as sns

In [4]:
data_dir = Path("/scratch/xref-experiments/")
vocabularies = load_vocabularies(data_dir / 'vocabulary')
data = ParquetModelData(data_dir / 'model-data', "r", vocabularies=vocabularies).open()

In [5]:
history = []
with open('../property_skipgram_100epoch_1024_history.json') as fd:
    history_data = json.loads(fd.read())
for phase, items in history_data.items():
    for metric, values in items.items():
        for epoch, value in enumerate(values):
            history.append({'phase': phase, 'metric': metric, 'value': value, 'epoch': epoch})
history = pd.DataFrame.from_dict(history)

In [6]:
model = PropertySkipgramModel(data, n_embed=1024)
model.load_state_dict(torch.load('../property_skipgram_100epoch_1024.tch'))

RuntimeError: Error(s) in loading state_dict for PropertySkipgramModel:
	size mismatch for property_embedding.embedding.weight: copying a param with shape torch.Size([74495, 1024]) from checkpoint, the shape in current model is torch.Size([70274, 1024]).

In [ ]:
history.head()

In [ ]:
g = sns.FacetGrid(history, col="metric")
g.map_dataframe(sns.lineplot, x='epoch', y='value', hue='phase')
g.add_legend()

In [ ]:
df = next(data.skipgrams('test', batch_size=65536))

In [ ]:
df.info()

In [ ]:
df['predict'] = model(test_sample).detach().numpy()
df['predict_class'] = (df['predict'] > 0.5).astype('int')
df['correct'] = (df['target'] == df['predict_class']).astype('int')

In [ ]:
pd.crosstab(df['target'], df['predict_class'], normalize='columns')

In [ ]:
g = sns.FacetGrid(df, col="schema_x", row='schema_y')
g.set_titles(col_template="{data.vocabularies['schema'].invert(col_name)}",
             row_template="{data.vocabularies['schema'].invert(row_name)}")
g.map_dataframe(sns.histplot, 'correct')
#g.add_legend()

In [ ]:
df.query('correct == 0')[['ngrams_x', 'ngrams_y']].applymap(data.vocabularies['ngrams'].invert).head(100)